In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv('C:\\Users\\SEC\\Desktop\\Movie Review-RNN\\IMDB Dataset.csv')


In [6]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [7]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [10]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [11]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [12]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.build(input_shape=(None, 200))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 128)          640000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 673025 (2.57 MB)
Trainable params: 673025 (2.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [17]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5


500/500 [==============================] - 54s 101ms/step - loss: 0.6913 - accuracy: 0.5366 - val_loss: 0.6633 - val_accuracy: 0.6190
Epoch 2/5
500/500 [==============================] - 55s 109ms/step - loss: 0.6072 - accuracy: 0.6530 - val_loss: 0.4688 - val_accuracy: 0.7851
Epoch 3/5
500/500 [==============================] - 55s 109ms/step - loss: 0.5089 - accuracy: 0.7558 - val_loss: 0.6457 - val_accuracy: 0.6314
Epoch 4/5
500/500 [==============================] - 57s 114ms/step - loss: 0.5689 - accuracy: 0.7002 - val_loss: 0.6612 - val_accuracy: 0.5910
Epoch 5/5
500/500 [==============================] - 55s 111ms/step - loss: 0.6211 - accuracy: 0.6420 - val_loss: 0.6229 - val_accuracy: 0.6361


In [18]:
model.save("model.h5")

In [19]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [20]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 [==============================] - 6s 17ms/step - loss: 0.6243 - accuracy: 0.6416


In [21]:
print(loss)

0.6243459582328796


In [22]:
print(accuracy)

0.6416000127792358


In [23]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [24]:
predictive_system("This movie was fantastic and amazing")

1/1 [==============================] - 0s 331ms/step


'positive'

In [25]:
predictive_system("A trilling adventure with stunning visual")

1/1 [==============================] - 0s 31ms/step


'positive'

In [26]:
predictive_system("A visual masterpiece")

1/1 [==============================] - 0s 29ms/step


'positive'

In [27]:
predictive_system("Overall long and slow")

1/1 [==============================] - 0s 36ms/step


'negative'